In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud_operations import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "CS340module3"
shelter = AnimalShelter(username, password)

# Query MongoDB and create DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Remove the '_id' column to avoid issues in Dash DataTable
df.drop(columns=['_id'], inplace=True, errors='ignore')

# Debugging: Print only the first 5 rows to avoid excessive output
print("DataFrame preview (first 5 rows):")
print(df.head(5))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i} for i in df.columns  # Automatically populate column names
        ],
        data=df.to_dict('records'),  # Display data from MongoDB
        row_selectable='single',  # Allow single-row selection
        page_size=10,  # Display 10 rows at a time
        selected_rows=[0]  # Select the first row by default
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Handle case where selected_columns is None
    if not selected_columns:
        return []  # Return an empty style if no columns are selected

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This function creates the geolocation map based on the selected row
def update_map(viewData, index):
    if not viewData or len(viewData) == 0:
        print("viewData is empty")
        return [html.P("No data available for map")]

    # Convert viewData to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Debugging: Print only the first 5 rows of the DataFrame
    print("Converted DataFrame preview (first 5 rows):")
    print(dff.head(5))

    if dff.empty:
        return [html.P("No data available for map")]

    # Handle selected row index
    if index is None or len(index) == 0:
        row = 0  # Default to the first row
    else:
        row = index[0]

    try:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),  # Animal breed
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(f"Name: {dff.iloc[row, 9]}"),  # Animal name
                                  html.P(f"Unique ID: {dff.iloc[row, 0]}")  # Unique Identifier
                              ])
                          ])
            ])
        ]
    except Exception as e:
        print("Error in update_map:", e)
        return [html.P("Error generating map")]

# This callback updates the map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),  # Update the map container
    [Input('datatable-id', "derived_virtual_data"),  # Data from the DataTable
     Input('datatable-id', "derived_virtual_selected_rows")]  # Selected row index
)
def update_map_callback(viewData, selected_rows):
    # Debugging: Print a sample of the data passed to the callback
    if viewData:
        print("viewData sample (first 5 rows):")
        print(viewData[:5])  # Print only the first 5 rows of viewData
    else:
        print("viewData is empty or None")

    print("selected_rows:", selected_rows)  # Debugging statement for selected rows
    return update_map(viewData, selected_rows)

# Run the server
app.run_server(debug=True)

In [ ]:
from crud_operations import AnimalShelter
print("Module imported successfully!")

In [ ]:
from pymongo import MongoClient

# MongoDB connection parameters
HOST = "nv-desktop-services.apporto.com"
PORT = 33257
USERNAME = "aacuser"
PASSWORD = "CS340module3"

try:
    # Create the MongoDB client
    client = MongoClient(f'mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}')
    
    # List databases to confirm connection
    databases = client.list_database_names()
    print("Connection successful! Databases available:", databases)
except Exception as e:
    print("Error connecting to MongoDB:", e)

In [ ]:
# Access the AAC database and animals collection
db = client["AAC"]
collection = db["animals"]

# Count documents in the collection
document_count = collection.count_documents({})
print(f"Number of documents in 'animals' collection: {document_count}")